## Avaliações: Metrica kappa aplicada à rotulação manual de 3 rotuladores

In [3]:
import pandas as pd
import numpy as np
# Importando a biblioteca utilsDS que possuí funções auxiliares compartilhadas
# com outros notebooks, exemplo: jsonl2list, merge, tokenizeAn 
from utilsDS import *

In [4]:
def cohen_kappa(ann1, ann2):
    """Computes Cohen kappa for pair-wise annotators.
    :param ann1: annotations provided by first annotator
    :type ann1: list
    :param ann2: annotations provided by second annotator
    :type ann2: list
    :rtype: float
    :return: Cohen kappa statistic
    """
    count = 0
    for an1, an2 in zip(ann1, ann2):
        if an1 == an2:
            count += 1
    A = count / len(ann1)  # observed agreement A (Po)

    uniq = set(ann1 + ann2)
    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = ann1.count(item)
        cnt2 = ann2.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E += count

    return round((A - E) / (1 - E), 4)

In [5]:
def fleiss_kappa(M):
    """Computes Fleiss' kappa for group of annotators.
    :param M: a matrix of shape (:attr:'N', :attr:'k') with 'N' = number of subjects and 'k' = the number of categories.
        'M[i, j]' represent the number of raters who assigned the 'i'th subject to the 'j'th category.
    :type: numpy matrix
    :rtype: float
    :return: Fleiss' kappa score
    """
    N, k = M.shape  # N is # of items, k is # of categories
    n_annotators = float(np.sum(M[0, :]))  # # of annotators
    tot_annotations = N * n_annotators  # the total # of annotations
    category_sum = np.sum(M, axis=0)  # the sum of each category over all items
    # chance agreement
    p = category_sum / tot_annotations  # the distribution of each category over all annotations
    PbarE = np.sum(p * p)  # average chance agreement over all categories

    # observed agreement
    P = (np.sum(M * M, axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))
    Pbar = np.sum(P) / N  # add all observed agreement chances per item and divide by amount of items

    return round((Pbar - PbarE) / (1 - PbarE), 4)

In [6]:
DATA_PATH = "./dados/"

In [7]:
#Lendo dados
dadosArthurFranco = tokenizeAn(merge(jsonl2list(DATA_PATH + "arthur_unknown.jsonl"), jsonl2list(DATA_PATH + "arthur.jsonl")))[0]
dadosThiagoSalles = tokenizeAn(merge(jsonl2list(DATA_PATH + "thiago_unknown.jsonl"), jsonl2list(DATA_PATH + "thiago.jsonl")))[0]
## Tokenizar os dados da Priscilla
dadosPriscillSilva = tokenizeAn(merge2(jsonl2list(DATA_PATH + "priscilla_unknown.jsonl"), jsonl2list(DATA_PATH + "priscilla.jsonl"), merge(jsonl2list(DATA_PATH + "thiago_unknown.jsonl"), jsonl2list(DATA_PATH + "thiago.jsonl"))))[0]

In [8]:
## palavras contém os tokens das sentenças que foram tokenizadas, dos arquivos de entradas.
palavras = [i[0] for i in dadosArthurFranco]

df = pd.DataFrame({
     'palavrasArthur': dadosArthurFranco,
     'palavrasThiago': dadosThiagoSalles,
     'palavrasPriscill': dadosPriscillSilva
    })

## Pega as rotulações feitas por cada pessoa
rotulacoesArthurFranco = [i[1] for i in dadosArthurFranco]
rotulacoesPriscillSilva = [i[1] for i in dadosPriscillSilva]
rotulacoesThiagoSalles = [i[1] for i in dadosThiagoSalles]

df = pd.DataFrame({'palavras': palavras,
     'Arthur': rotulacoesArthurFranco,
     'Thiago': rotulacoesThiagoSalles,
     'Priscilla': rotulacoesPriscillSilva
    })


# Tirando palavras nao anotadas por ninguem 
dados = zip(palavras, rotulacoesArthurFranco, rotulacoesThiagoSalles, rotulacoesPriscillSilva)
semNone = [x for x in list(dados) if (x[1], x[2], x[3]) != ('o', 'o', 'o')]
palavraSNones, rotulacoesSNoneArthurFranco, rotulacoesSNoneThiagoSalles, rotulacoesSNonePriscillSilva = zip(*semNone)

df = pd.DataFrame({'palavras': palavraSNones,
     'Arthur': rotulacoesSNoneArthurFranco,
     'Thiago': rotulacoesSNoneThiagoSalles,
     'Priscilla': rotulacoesSNonePriscillSilva
    })

gerarCSV(DATA_PATH + "dsComparacaoRotulacaoSemTokenNaoRotulados", df)

In [9]:
## Calcula a metrica cohen

dados = zip(rotulacoesArthurFranco, rotulacoesThiagoSalles)
semNone = [x for x in list(dados) if x != ('o', 'o')]
arthur, thiago = zip(*semNone)
print(cohen_kappa(arthur,thiago))

0.8748


In [10]:
categorias = ["NOME_BEBIDA", "GRADUACAO_ALCOOLICA", "EQUIPAMENTO_DESTILACAO",
            "TEMPO_ARMAZENAMENTO", "RECIPIENTE_ARMAZENAMENTO", "TIPO_MADEIRA",
            "CARACTERISTICA_SENSORIAL_COR", "CARACTERISTICA_SENSORIAL_AROMA", 
            "CARACTERISTICA_SENSORIAL_SABOR", "CARACTERISTICA_SENSORIAL_CONSISTÊNCIA", 
            "NOME_PESSOA", "NOME_LOCAL", "NOME_ORGANIZACAO", "TEMPO", "PRECO", "VOLUME", "CLASSIFICACAO_BEBIDA", "o"]

In [11]:
## Calcula a metrica Fleiss desconsiderando tokens não rotuladas por nenhum rotulador

dados = zip(rotulacoesArthurFranco, rotulacoesThiagoSalles, rotulacoesPriscillSilva)
semNone = [x for x in list(dados) if x != ('o', 'o', 'o')]
arthur, thiago, priscilla = zip(*semNone)

M = np.zeros((len(arthur), len(categorias)))

nLines, nColumns = M.shape

for i in range(nLines):
    # if (arthur[i] != "o"):
        M[i, categorias.index(arthur[i])] += 1
    # if (thiago[i] != "o"):
        M[i, categorias.index(thiago[i])] += 1
    # if (priscilla[i] != "o"):
        M[i, categorias.index(priscilla[i])] += 1


print(fleiss_kappa(M))

0.8575


In [12]:
## Calcula a metrica Fleiss considerando tokens não rotulados por nenhum rotulador

M_com_None = np.zeros((len(rotulacoesArthurFranco), len(categorias)))

nLines, nColumns = M_com_None.shape

for i in range(nLines):
    M_com_None[i, categorias.index(rotulacoesArthurFranco[i])] += 1
    M_com_None[i, categorias.index(rotulacoesThiagoSalles[i])] += 1
    M_com_None[i, categorias.index(rotulacoesPriscillSilva[i])] += 1


print(fleiss_kappa(M_com_None))

0.9203
